<a href="https://colab.research.google.com/github/AUT-Student/NLP-HW3/blob/main/NLP_HW3_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [142]:
!pip install bert_embedding

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.14.6-cp37-cp37m-manylinux1_x86_64.whl (13.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0+zzzcolab20220506162203 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
yellowbrick 1.4 requires numpy>=1.16.0, but you have numpy 1.14.6 which is incompatible.
xarray 0.20.2 requires numpy>=1.18, but you have numpy 1.14.6 which is incompatible.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.14.6 which is incompatible.
tifffile 2021.11.2 requires numpy>=1.15.1, but you have numpy 1.14.6 which is incompatible.
tensorflow 2.8.0+zzzcolab2022

In [143]:
import pandas as pd
import numpy as np

from bert_embedding import BertEmbedding

# Dataset 

In [144]:
!gdown 1oin_Sw1Gk_WLS9zpDrap5FdlRcUrTP_D
!gdown 1EvtGQ8-sYXXQ3VA9ByjD4OmFH13WxvAs
!gdown 154f-z0PsPAp0yvOLdNXP8NXYjPljpgCZ
!gdown 1EDG_j6F5ohIjpkihRQd0-i9daHWV7StN

Downloading...
From: https://drive.google.com/uc?id=1oin_Sw1Gk_WLS9zpDrap5FdlRcUrTP_D
To: /content/Sentenses_train.txt
100% 1.23M/1.23M [00:00<00:00, 148MB/s]
Downloading...
From: https://drive.google.com/uc?id=1EvtGQ8-sYXXQ3VA9ByjD4OmFH13WxvAs
To: /content/Senses_train.txt
100% 49.7k/49.7k [00:00<00:00, 39.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=154f-z0PsPAp0yvOLdNXP8NXYjPljpgCZ
To: /content/Sentenses_test.txt
100% 213k/213k [00:00<00:00, 71.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1EDG_j6F5ohIjpkihRQd0-i9daHWV7StN
To: /content/Senses_test.txt
100% 8.76k/8.76k [00:00<00:00, 8.36MB/s]


In [145]:
def create_dataset(part_name):
  sentences_file = open(f"/content/Sentenses_{part_name}.txt", "r")
  senses_file = open(f"/content/Senses_{part_name}.txt", "r")

  sentences = sentences_file.readlines()
  senses = senses_file.readlines()
  
  dataset_sentence = []
  dataset_sense = []
  dataset_ambiguity_word = []

  for sentence, sense in list(zip(sentences, senses)):
    dataset_sense.append(sense[:-1])
    tokens = sentence.split()
    word_index = tokens.index("<head>")
    dataset_ambiguity_word.append(tokens[word_index+1].lower())
    dataset_sentence.append(" ".join(
        tokens[max(0, word_index-11):word_index-1] +\
        [tokens[word_index+1]] +\
        tokens[word_index+3:min(word_index+13, len(tokens))])
    )
  
  return pd.DataFrame({"Sentence": dataset_sentence, "Ambiguity_Word": dataset_ambiguity_word, "Sense": dataset_sense})

In [146]:
train_dataset = create_dataset("train")
test_dataset = create_dataset("test")

# Embedding

In [147]:
# GPU
# ctx = mx.gpu(0)
# bert_embedding = BertEmbedding(ctx=ctx)

# CPU
bert_embedding = BertEmbedding()

In [149]:
def calculate_ambiguity_vector(dataset):
  dataset["Ambiguity_Vector"] = None
  for i, data in dataset.iterrows():
    if i%300==0: print(f"{int(i*100/len(dataset))}%")
    tokens, vectors = bert_embedding([data["Sentence"]])[0]
    ambiguity_word = data["Ambiguity_Word"]

    try:    
      ambiguity_vector = vectors[tokens.index(ambiguity_word)]
    except ValueError:
      tokens, vectors = bert_embedding([data["Sentence"][15:]])[0]
      ambiguity_word = data["Ambiguity_Word"]
      ambiguity_vector = vectors[tokens.index(ambiguity_word)]

    dataset.at[i, "Ambiguity_Vector"] = ambiguity_vector

In [150]:
calculate_ambiguity_vector(train_dataset)

0%
4%
9%
14%
18%
23%
28%
33%
37%
42%
47%
52%
56%
61%
66%
71%
75%
80%
85%
90%
94%
99%


In [ ]:
calculate_ambiguity_vector(test_dataset)

0%
26%
